In [1]:
import os
import pandas as pd
import gzip
import microhomology as micro
import pickle


#MGYG000000003_1	Prodigal:2.6	CDS	23749	24648	.	-	0	ID=MGYG000000003_00025;Name=fieF;db_xref=COG:COG0053;gene=fieF;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:P69380;locus_tag=MGYG000000003_00025;product=Ferrous-iron efflux pump FieF;eggNOG=679935.Alfi_1785;COG=P;KEGG=-;Pfam=PF01545;InterPro=IPR027469,IPR002524
#MGYG000000003_1	Prodigal:2.6	CDS	24855	26339	.	-	0	ID=MGYG000000003_00026;eC_number=6.1.1.15;Name=proS;db_xref=COG:COG0442;gene=proS;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:Q5SM28;locus_tag=MGYG000000003_00026;product=Proline--tRNA ligase;eggNOG=679935.Alfi_1784;COG=J;KEGG=ko:K01881;Pfam=PF00587,PF03129,PF09180;InterPro=IPR016061,IPR017449,IPR004154,IPR002314,IPR033721,IPR036621,IPR006195,IPR004499

infile = '../../HGT_demo_file/SAMEA3449210.event_output.csv'
db_idir = '../../HGT_demo_file/DB.genome_annotation'
pfile = '../../HGT_demo_file/ko_pathway_dict.pickle'
outdir = '.'
fr_size = 1000
trans = micro.Transfer_times()
trans.read_events(infile)


sample num  1 kept HGTs num is 10


In [4]:
trans.HGT_event_dict['SAMEA3449210'][0]

In [ ]:

df = pd.read_csv(infile, header=0, index_col=None)
df.rename(columns={'receptor':'recipient'}, inplace=True)

In [21]:
def gz2df(ifile):
    with gzip.open(ifile, 'rb') as f:
        df = pd.read_csv(f, header=None, index_col=None, sep='\t')
    return df

def extract_line(content):
    ko_list = []
    cog_list = []
    cog_cate_list = []
    # check kegg
    if 'KEGG=' in content:
        kegg = content.split('KEGG=')[1].strip().split(';')[0]
        kos = kegg.split(',')
        #print('kos', kos)
        for ko in kos:
            if ko.startswith('ko:'):
                ko = ko.split('ko:')[-1]
                ko_list.append(ko)
    
    if 'db_xref=' in content:
        cog = content.split('db_xref=')[1].strip().split(';')[0]
        cogs = cog.split(',')
        for cog in cogs:
            if cog.startswith('COG:'):
                cog = cog.split('COG:')[-1]
                cog_list.append(cog)

    if 'COG=' in content:
        cog_cate = content.split('COG=')[1].strip().split(';')[0]
        cog_cate = cog_cate.split(',')[0]
        for cc in cog_cate:
            if cc != '-':
                cog_cate_list += [char for char in cc]
    return ko_list, cog_list, cog_cate_list

def extract(df):
    ko_list = []
    cog_list = []
    cog_cate_list = []
    for idx in df.index:
        kos, cogs, cog_cates = extract_line(df.loc[idx, 8])
        ko_list += kos
        cog_list += cogs
        cog_cate_list += cog_cates
    return ko_list, cog_list, cog_cate_list

def overlap(range1, range2):
    if range1[0] > range2[1] or range1[1] < range2[0]:
        return False
    else:
        return True

# styp = recipient or donor
def search_event(scaffold, db, range):
    scaffold_df = db[db[0] == scaffold]
    valid_idx = []
    for idx in scaffold_df.index:
        if overlap(range, [scaffold_df.loc[idx, 3], scaffold_df.loc[idx, 4]]):
            valid_idx.append(idx)
    valid_df = scaffold_df.loc[valid_idx, ]
    return valid_df
    
def search_row(idx, df, db_dir, fr_size):
    tmp = '{}.gff.gz'
    row = df.loc[idx, ]
    # for recipient
    recipient = row['recipient']
    chrom = recipient.split('_')[0]
    ifile = os.path.join(db_dir, tmp.format(chrom))
    db = gz2df(ifile)
    recipient_range = [max(0, row['insert_locus']-fr_size), row['insert_locus']+fr_size]
    recipient_df = search_event(recipient, db, recipient_range)
    recipient_excluded_df = get_backgroud(recipient, db, recipient_range)
    # for donor
    donor = row['donor']
    range = [max(0, row['delete_start'] - fr_size), row['delete_end'] + fr_size]
    chrom = donor.split('_')[0]
    ifile = os.path.join(db_dir, tmp.format(chrom))
    db = gz2df(ifile)
    donor_df = search_event(donor, db, range)
    donor_excluded_df = get_backgroud(donor, db, range)
    return recipient_df, donor_df, recipient_excluded_df, donor_excluded_df 

def get_backgroud(scaffold, db, range_excluded):
    scaffold_df = db[db[0] == scaffold]
    valid_idx = []
    for idx in scaffold_df.index:
        if not overlap(range_excluded, [scaffold_df.loc[idx, 3], scaffold_df.loc[idx, 4]]):
            valid_idx.append(idx)
    valid_df = scaffold_df.loc[valid_idx, ]
    return valid_df

In [23]:
result_anno = pd.DataFrame(columns=['id', 'sample', 'recipient_KEGG_n', 'recipient_KEGG_list', 'recipient_COG_n', 'recipient_COG_list',
                                    'donor_KEGG_n', 'donor_KEGG_list', 'donor_COG_n', 'donor_COG_list',
                                    'recipient', 'insert_locus', 'donor', 'delete_start', 'delete_end', 'reverse_flag'])

bk_ko_set = set()
bk_cog_set = set()
exist_ko_set = set()
exist_cog_set = set()

for idx in df.index:
    recipient_df, donor_df, recipient_excluded_df, donor_excluded_df = search_row(idx, df, db_idir, fr_size)
    id = 'HGT_c{}'.format(idx+1)
    sample = df.loc[idx, 'sample']
    ko_list, cog_list, cog_cate = extract(recipient_df)
    exist_ko_set.update(set(ko_list))
    exist_cog_set.update(set(cog_cate))
    recipient_KEGG_n = len(set(ko_list))
    recipient_COG_n = len(set(cog_list))
    recipient_KEGG_list = ';'.join(set(ko_list))
    recipient_COG_list = ';'.join(set(cog_list))
    if recipient_KEGG_n == 0:
        recipient_KEGG_list = 'NA'
    if recipient_COG_n == 0:
        recipient_COG_list = 'NA'

    ko_list, cog_list, cog_cate = extract(donor_df)
    exist_ko_set.update(set(ko_list))
    exist_cog_set.update(set(cog_cate))
    donor_KEGG_n = len(set(ko_list))
    donor_COG_n = len(set(cog_list))
    donor_KEGG_list = ';'.join(set(ko_list))
    donor_COG_list = ';'.join(set(cog_list))
    if donor_KEGG_n == 0:
        donor_KEGG_list = 'NA'
    if donor_COG_n == 0:
        donor_COG_list = 'NA'
    recipient = df.loc[idx, 'recipient']
    insert_locus = df.loc[idx, 'insert_locus']
    donor = df.loc[idx, 'donor']
    delete_start = df.loc[idx, 'delete_start']
    delete_end = df.loc[idx, 'delete_end']
    reverse_flag = df.loc[idx, 'reverse_flag']
    result_anno.loc[len(result_anno), ] = [id, sample, recipient_KEGG_n, recipient_KEGG_list, recipient_COG_n, recipient_COG_list, donor_KEGG_n, donor_KEGG_list, donor_COG_n, donor_COG_list, recipient, insert_locus, donor, delete_start, delete_end, reverse_flag]

    # add bg ko
    ko_list, cog_list, cog_cate = extract(recipient_excluded_df)
    bk_ko_set.update(set(ko_list))
    bk_cog_set.update(set(cog_cate))

    ko_list, cog_list, cog_cate = extract(donor_excluded_df)
    bk_ko_set.update(set(ko_list))
    bk_cog_set.update(set(cog_cate))
    
result_anno.to_csv(os.path.join(outdir, 'output.functional_annotation.annotated.tsv'), index=False, sep='\t')


In [18]:
background_counts = ke.get_pathways(list(bk_ko_set), ko_pathway_dict)
input_counts = ke.get_pathways(list(exist_ko_set), ko_pathway_dict)
opath = os.path.join(outdir, 'output.functional_annotation.enrich.KEGG.tsv')
ke.enrichment_analysis(list(exist_ko_set), list(bk_ko_set), input_counts, background_counts, opath)
opath = os.path.join(outdir, 'output.functional_annotation.enrich.COG.tsv')
ce.cog_enrich(opath, list(exist_cog_set), list(bk_cog_set) )


KeyboardInterrupt: 